In [1]:
from wordcloud import WordCloud

# Manual PreProcessing (Lower Case All)

In [2]:
#make all lowercase
def lower_df_text(df, text_columns):
    for column in text_columns:
        df[column] = df[column].apply(lambda x: x.lower())
    return

# Generating Wordclouds

In [3]:
def plot_wordcloud(text, figname='wordcloud', rgb=(255,255,255)):
    font_path = "/Library/Fonts/DIN Condensed Bold.ttf"
    wc = WordCloud(background_color="white",max_words=25,
                   collocations=False, font_path=font_path, scale=5, color_func=lambda *args, **kwargs: rgb)
    wc.generate(text)
    
    fig, ax = plt.subplots(figsize=(14,18))
    plt.imshow(wc)
    plt.savefig('data/wordclouds/{}'.format(figname), dpi=240)


# Generating Network Graphs


In [4]:
import networkx as nx
from networkx.algorithms import community

In [5]:
# Functionize

def generate_graph(edge_filepath, node_filepath, title='gephi_graph'):
    edges = pd.read_csv(edge_filepath, header=None)
    nodes = pd.read_csv(node_filepath, header=None)
    edges.columns = ['Source', 'Target']
    nodes.columns = ['Node', 'Alias']
    nodes = nodes.astype('str')
    edges = edges.astype('str')
    edge_tuples = []
    for source, target in zip(edges['Source'], edges['Target']):
        edge_tuples.append((source, target))
    G = nx.DiGraph()
    G.add_nodes_from(nodes['Node'])
    G.add_edges_from(edge_tuples)
    print(nx.info(G))
    nx.write_gexf(G, edge_filepath.append('{}'.format(title)))
    

# Text Processing Pipeline (Too Slow)
from https://www.kaggle.com/balatmak/text-preprocessing-steps-and-universal-pipeline#Reusable-pipeline

In [6]:
import numpy as np
import multiprocessing as mp

import string
import spacy 
import en_core_web_sm
from nltk.tokenize import word_tokenize
from sklearn.base import TransformerMixin, BaseEstimator
from normalise import normalise

nlp = en_core_web_sm.load()


class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self,
                 variety="BrE",
                 user_abbrevs={},
                 n_jobs=1, custom_stop_words=[]):
        """
        Text preprocessing transformer includes steps:
            1. Text normalization
            2. Punctuation removal
            3. Stop words removal
            4. Lemmatization
        
        variety - format of date (AmE - american type, BrE - british format) 
        user_abbrevs - dict of user abbreviations mappings (from normalise package)
        n_jobs - parallel jobs to run
        """
        self.variety = variety
        self.user_abbrevs = user_abbrevs
        self.n_jobs = n_jobs
        self.custom_stop_words=custom_stop_words

    def fit(self, X, y=None):
        return self

    def transform(self, X, *_):
        X_copy = X.copy()

        partitions = 1
        cores = mp.cpu_count()
        if self.n_jobs <= -1:
            partitions = cores
        elif self.n_jobs <= 0:
            return X_copy.apply(self._preprocess_text)
        else:
            partitions = min(self.n_jobs, cores)

        data_split = np.array_split(X_copy, partitions)
        pool = mp.Pool(cores)
        data = pd.concat(pool.map(self._preprocess_part, data_split))
        pool.close()
        pool.join()

        return data

    def _preprocess_part(self, part):
        return part.apply(self._preprocess_text)

    def _preprocess_text(self, text):
        normalized_text = self._normalize(text)
        doc = nlp(normalized_text)
        removed_punct = self._remove_punct(doc)
        removed_stop_words = self._remove_stop_words(removed_punct)
        return self._lemmatize(removed_stop_words)

    def _normalize(self, text):
        # some issues in normalise package
        try:
            return ' '.join(normalise(text, variety=self.variety, user_abbrevs=self.user_abbrevs, verbose=False))
        except:
            return text

    def _remove_punct(self, doc):
        return [t for t in doc if t.text not in string.punctuation]

    def _remove_stop_words(self, doc):
        return [t for t in doc if t.is_stop != True and t.text not in self.custom_stop_words  and 'https' not in t.text]

    def _lemmatize(self, doc):
        return ' '.join([t.lemma_ for t in doc])

/Users/jacobmullins/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.semi_supervised.label_propagation module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.semi_supervised. Anything that cannot be imported from sklearn.semi_supervised is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/jacobmullins/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelPropagation from version 0.18 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Democratic Convention Data


In [8]:
dconvention_df = pd.read_csv('large_data/dconvention-tweet-ids.csv')
dconvention_df.describe()
dconvention_df.columns

,favorite_count,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,reweet_id,retweet_screen_name,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count,user_time_zone
count,134687.000000,1.346870e+05,7.540000e+03,1.724400e+04,134687.000000,0.0,0.0,1.346870e+05,1.346870e+05,134687.000000,134687.000000,1.346870e+05,0.0
mean,3.345653,7.582166e+17,7.576638e+17,1.624101e+16,1.681061,NaN,NaN,3.216183e+04,9.140628e+03,2767.032609,133.059946,4.797842e+04,NaN
std,93.861568,5.226539e+14,1.226524e+16,1.073660e+17,60.445901,NaN,NaN,6.600100e+04,9.132616e+04,7243.250845,484.443211,9.282556e+04,NaN
min,0.000000,7.564538e+17,4.238422e+16,1.200000e+01,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,1.000000e+00,NaN
25%,0.000000,7.577726e+17,7.576961e+17,1.651195e+07,0.000000,NaN,NaN,2.686000e+03,3.920000e+02,520.000000,11.000000,6.261000e+03,NaN
50%,0.000000,7.581443e+17,7.580650e+17,5.524592e+07,0.000000,NaN,NaN,1.019400e+04,1.110000e+03,1166.000000,34.000000,1.779900e+04,NaN
75%,1.000000,7.586317e+17,7.584846e+17,5.266899e+08,0.000000,NaN,NaN,3.172300e+04,3.352000e+03,2645.000000,97.000000,4.917500e+04,NaN
max,30492.000000,7.594183e+17,7.594173e+17,7.591036e+17,20768.000000,NaN,NaN,1.053126e+06,1.372433e+07,317437.000000,30229.000000,2.230300e+06,NaN


Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'reweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_screen_name', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name.1',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified'],
      dtype='object')

In [36]:
#create a node df with summary columns for each user
dnode_features = [
       'favorite_count', 'place',
       'retweet_count',
       'text', 
       'user_screen_name', 'user_description',
       'user_followers_count',
       'user_location'
       ]
dnode_df = dconvention_df[dnode_features].copy()
#fill in 'place' column if isNaN, using the location of the User.
nan_map = dnode_df['place'].isna()
dnode_df['place'][nan_map] = dnode_df['user_location'][nan_map]
#group by user_id and sum various metrics for the user
dnode_df = dnode_df.groupby(['user_screen_name', 'place', 'user_description']).sum()
dnode_df = dnode_df.reset_index().set_index('user_screen_name')
dnode_df

/Users/jacobmullins/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,place,user_description,favorite_count,retweet_count,user_followers_count
user_screen_name,,,,,
0000livia,"Suffolk, VA",Jesus Lover|Wife of @qfranklin23 | Mother | He...,1,0,546
01RedZ28,"Park Ridge, NJ","New York Giants, New York Rangers, New York Ya...",0,0,114
033sdn,"Boardman, OH","it was the age of wisdom, it was the age of fo...",0,0,1582
08meghana,"Palatine, IL",Mumbaite living in USA🌎,6,1,4030
0SamanthaRae0,"Port St John, FL","Im a mother, a sister and a daughter!♡",1,0,65
...,...,...,...,...,...
zwoebe,"Groningen, Nederland",Drinkt Krombacher met koerspetje op z'n kop. D...,1,0,711
zx10arlo,"Omaha, NE",Founder of Mav Motors at UNO. Car and Bike En...,3,0,265
zxcvanessa,"Oakley, CA","always forward, never back",0,0,394


In [63]:
#join the node data onto the mention nodes csv
dmentions = pd.read_csv('data/democrat/convention_mentions/dmention-nodes.csv', index_col='Label')
dmentions
dmentions.merge(dnode_df, left_index=True, right_index=True, how='left')
dmentions.nunique

,Id
Label,
weeklystandard,17546958
realDonaldTrump,25073877
HillaryClinton,1339835893
Natire2u,255645890
AnitaDWhitee,909448512
...,...
LatinoCommFdn,224810490
cspanMatthew,576520016
ShellsBells143,616692598


,Id,place,user_description,favorite_count,retweet_count,user_followers_count
01elibo,2275927727,NaN,NaN,NaN,NaN,NaN
09072021,388150165,NaN,NaN,NaN,NaN,NaN
0Sundance,425116151,NaN,NaN,NaN,NaN,NaN
1,18899974,NaN,NaN,NaN,NaN,NaN
100DaysOfHRC,758675398913765376,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
zstollar,620473700,NaN,NaN,NaN,NaN,NaN
ztsamudzi,2968635633,NaN,NaN,NaN,NaN,NaN
zuma02,221642866,NaN,NaN,NaN,NaN,NaN
zxnaida,1070209219,NaN,NaN,NaN,NaN,NaN


<bound method DataFrame.nunique of                          Id
Label                      
weeklystandard     17546958
realDonaldTrump    25073877
HillaryClinton   1339835893
Natire2u          255645890
AnitaDWhitee      909448512
...                     ...
LatinoCommFdn     224810490
cspanMatthew      576520016
ShellsBells143    616692598
RevJacquiLewis    587591389
RevKHenderson    1289348384

[15737 rows x 1 columns]>

# Republican Convention Data 

In [9]:
rconvention_df = pd.read_csv('large_data/rconvention-tweet-ids.csv')
rconvention_df.describe()
rconvention_df.columns

,favorite_count,id,in_reply_to_status_id,in_reply_to_user_id,retweet_count,reweet_id,retweet_screen_name,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_statuses_count,user_time_zone
count,113631.000000,1.136310e+05,5.650000e+03,1.241000e+04,113631.000000,0.0,0.0,1.136310e+05,1.136310e+05,113631.000000,113631.000000,1.136310e+05,0.0
mean,3.855304,7.556725e+17,7.549740e+17,1.055017e+16,1.846283,NaN,NaN,3.264499e+04,1.316484e+04,2666.006187,150.543109,4.795156e+04,NaN
std,117.449660,4.707778e+14,1.442378e+16,8.699262e+16,52.296739,NaN,NaN,6.640411e+04,4.817132e+05,7370.316474,901.512807,8.528117e+04,NaN
min,0.000000,7.539204e+17,2.325725e+17,1.200000e+01,0.000000,NaN,NaN,0.000000e+00,0.000000e+00,0.000000,0.000000,2.000000e+00,NaN
25%,0.000000,7.552497e+17,7.552227e+17,1.891643e+07,0.000000,NaN,NaN,2.916500e+03,3.950000e+02,545.000000,11.000000,7.076000e+03,NaN
50%,0.000000,7.555901e+17,7.555728e+17,4.006901e+07,0.000000,NaN,NaN,1.036500e+04,1.162000e+03,1188.000000,35.000000,1.965900e+04,NaN
75%,1.000000,7.560576e+17,7.559575e+17,2.541900e+08,0.000000,NaN,NaN,3.264450e+04,3.377000e+03,2717.000000,104.000000,5.337700e+04,NaN
max,26220.000000,7.564522e+17,7.564517e+17,7.562792e+17,11502.000000,NaN,NaN,1.053127e+06,7.967907e+07,508859.000000,116472.000000,1.611752e+06,NaN


Index(['coordinates', 'created_at', 'hashtags', 'media', 'urls',
       'favorite_count', 'id', 'in_reply_to_screen_name',
       'in_reply_to_status_id', 'in_reply_to_user_id', 'lang', 'place',
       'possibly_sensitive', 'retweet_count', 'reweet_id',
       'retweet_screen_name', 'source', 'text', 'tweet_url', 'user_created_at',
       'user_screen_name', 'user_default_profile_image', 'user_description',
       'user_favourites_count', 'user_followers_count', 'user_friends_count',
       'user_listed_count', 'user_location', 'user_name', 'user_screen_name.1',
       'user_statuses_count', 'user_time_zone', 'user_urls', 'user_verified'],
      dtype='object')

# Democratic Candidates Data

# Republican Candidates Data

In [64]:
import preprocessor as p
p.clean('preprocessing and it’s always better to be sure that our data is cleaned 100%.')

'preprocessing and it’s always better to be sure that our data is cleaned %.'